In [16]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [17]:
%hive_start

In [18]:
%timeout 300

In [19]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [20]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

Deleted /tmp/output
-rw-r--r--   1 root supergroup        637 2020-01-20 02:49 /tmp/output/data.tsv


In [21]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;

DROP TABLE IF EXISTS data;
OK
Time taken: 16.758 seconds
CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 1.876 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;
Loading data to table default.data
OK
Time taken: 2.582 seconds


In [22]:
%%hive
--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
SELECT numero 
    FROM (select distinct numero from data order by numero) w LIMIT 5;

--
-- para guardar la consulta se debe almacenar en un tabla
DROP TABLE IF EXISTS salida;
OK
Time taken: 1.068 seconds
CREATE TABLE salida
AS
SELECT numero 
    FROM (select distinct numero from data order by numero) w LIMIT 5;
Query ID = root_20200120025018_3de9b9fc-de15-4b20-aaf2-137d80e72cbe
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579488152080_0004, Tracking URL = http://684b85a9e2af:8088/proxy/application_1579488152080_0004/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579488152080_0004
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-20 02:50:37,103 Stage-1

In [23]:
%%hive
DROP TABLE IF EXISTS data;

DROP TABLE IF EXISTS data;
OK
Time taken: 0.198 seconds


In [24]:
%%hive
-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

-- 
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120025149_fde9dd39-7557-4469-967b-ef704de16272
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579488152080_0006, Tracking URL = http://684b85a9e2af:8088/proxy/application_1579488152080_0006/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579488152080_0006
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 02:52:02,124 Stage-1 map = 0%,  reduce = 0%
2020-01-20 02:52:11,811 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 4.06 sec
MapReduce Total cumulative CPU time: 4 seconds 60 msec
Ended Job = job_1579488152080_0006
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/0000

In [25]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [26]:
%hive_quit